# 1920年の東京市と2020年の東京都で, 8月の平均気温に差があるか検定する.

但し, 使用するデータは両者とも1日から20日までとする.

In [3]:
import math
import pandas as pd
import numpy as np
from scipy import stats

import warnings
warnings.filterwarnings('ignore')

## 標本平均の計算と比較
---

国土交通省気象庁にて公開されている標本データを使用.<br><br>
[1920年8月のデータ](https://www.data.jma.go.jp/obd/stats/etrn/view/daily_s1.php?prec_no=44&block_no=47662&year=1920&month=8&day=&view=p1)<br>
[2020年8月のデータ](https://www.data.jma.go.jp/obd/stats/etrn/view/daily_s1.php?prec_no=44&block_no=47662&year=2020&month=8&day=&view=p1)

In [43]:
# 1920年8月1日~20日の東京市の平均気温(℃)
data_1920 = pd.Series([27.2, 26.3, 26.8, 26.2, 25.2, 24.6, 24.3, 25.1, 26.2, 25.1,
                       25.3, 26.2, 26.1, 27.2, 27.1, 26.2, 26.2, 25.1, 24.2, 26.3])

# 2020年8月1日~20日の東京都の平均気温(℃)
data_2020 = pd.Series([26.1, 26.3, 27.2, 28.3, 29.1, 28.5, 29.5, 28.1, 28.7, 30.5,
                       31.7, 30.0, 29.4, 29.4, 31.0, 31.2, 30.8, 30.3, 29.0, 29.7])


print(f"1920年8月の平均気温 = {data_1920.mean():.2f}")
print(f"2020年8月の平均気温 = {data_2020.mean():.2f}")
print(f"1920年・2020年の8月における平均気温の差 = {(abs(data_1920.mean() - data_2020.mean())):.2f}")

1920年8月の平均気温 = 25.84
2020年8月の平均気温 = 29.24
1920年・2020年の8月における平均気温の差 = 3.40


## 母集団の推定

In [44]:
print(f"1920年のデータの母集団の平均推定値 = {data_1920.mean():.2f}")
print(f"2020年のデータの母集団の平均推定値 = {data_2020.mean():.2f}")
print(f"1920年のデータの母集団の標準偏差の推定値(不偏標準偏差) = {data_1920.std(ddof=1):.2f}")
print(f"2020年のデータの母集団の標準偏差の推定値(不偏標準偏差) = {data_2020.std(ddof=1):.2f}")

1920年のデータの母集団の平均推定値 = 25.84
2020年のデータの母集団の平均推定値 = 29.24
1920年のデータの母集団の標準偏差の推定値(不偏標準偏差) = 0.93
2020年のデータの母集団の標準偏差の推定値(不偏標準偏差) = 1.54


## 対応なしｔ検定の前提条件のチェック
---

平均気温に実質的な差があっても, それは偶然による誤差である可能性がある.<br>
この平均気温の差が偶然であるか否かを『対応なしｔ検定』によって定量的に見る.<br>

但し, 対応なしｔ検定を適用するにあたり, 標本について以下の条件を満たす必要がある.<br>

1. 2つの母集団から標本が無作為に抽出されていること.
2. 比較する2群のデータの母集団が正規分布, またはそれに近い分布となっていること(**正規姓**).
3. 比較する2群のデータの分散が等しいこと(**等分散性**).

## 正規性の検定
---

**シャピロ・ウィルク検定** (Shapiro-Wilk 検定) により正規性の検定を行う.<br><br>

シャピロ・ウィルク検定は「標本は正規分布に従う母集団から無作為抽出されたものである」を帰無仮説とする.<br>
今回は有意水準を5% (=0.05) とする.<br>
そして, 検定により求まる有意確率 p が5%よりも大きいならば「母集団は正規分布に従う」という仮説を採択する.<br>
ここでの有意確率 p の具体的な意味は「正規母集団から無作為抽出したときに今回の標本分布が観測される確率」を意味する.


In [45]:
# シャピロ・ウィルク検定
_, pa = stats.shapiro(data_1920)
_, pb = stats.shapiro(data_2020)

print(f"1920年のデータ p値 = {pa:.3f}")
print(f"2020年のデータ p値 = {pb:.3f}")

1920年のデータ p値 = 0.117
2020年のデータ p値 = 0.622


p 値はいずれも p >= 0.05 (5%) であるので, **帰無仮説を採択**して『各教本の母集団には正規性がある』と判断する.<br>

なお, この母集団が正規分布に従うということが言えると, 各標本の母集団について平均値の **区間推定** (母平均の区間推定) ができる.<br>
母平均の95%信頼区間を求める.

In [46]:
# 1920年のデータ
u2 = data_1920.var(ddof=1)           # 母集団の分散推定値(不偏分散)
m = data_1920.mean()                 # 標本平均
DF = len(data_1920) - 1              # 自由度
SE = math.sqrt(u2 / len(data_1920))  # 標準偏差
CI1, CI2 = stats.t.interval(alpha=0.95, loc=m, scale=SE, df=DF)

print(f"1920年のデータにおける母平均の95%信頼区間 CI = [{CI1:.2f}, {CI2:.2f}]")


# 2020年のデータ
u2 = data_2020.var(ddof=1)           
m = data_2020.mean()                 
DF = len(data_2020) - 1              
SE = math.sqrt(u2 / len(data_2020))  
CI1, CI2 = stats.t.interval(alpha=0.95, loc=m, scale=SE, df=DF)

print(f"2020年のデータにおける母平均の95%信頼区間 CI = [{CI1:.2f}, {CI2:.2f}]")

1920年のデータにおける母平均の95%信頼区間 CI = [25.41, 26.28]
2020年のデータにおける母平均の95%信頼区間 CI = [28.52, 29.96]


## 等分散性の検定
---

**ルビーン検定** (Levene 検定) により等分散性の検定を行う.<br>
この検定は, 2つの母分散が等しいことを帰無仮説とする.<br>
なお, ここでも有意水準5%で検定を行う.

In [47]:
# ルビーン検定
_, p = stats.levene(data_1920, data_2020, center="mean")

print(f"p 値 = {p:.3f}")

p 値 = 0.094


p 値 (優位確率) が p >= 0.05 (5%) であるので,帰無仮説を採択して『2つの標本には等分散性がある』と判断する.<br>


## スチューデントのｔ検定 (Student t-test)
---

対応なしｔ検定 (対応なしのスチューデントのｔ検定) として検定を行う。<br>

- スチューデントのｔ検定 (Student t-test) とは, 2群間の平均値の差を比較する検定のこと.
- 2群間の平均値が独立であり, 2群間に等分散性が仮定できる場合に用いる.<br><br>

『2つの標本の平均値に有意差がない』ことを帰無仮説とする.




In [48]:
# スチューデントのt検定
t, p = stats.ttest_ind(data_1920, data_2020, equal_var=True)
MU = abs(data_1920.mean() - data_2020.mean())
SE = MU / t
DF = len(data_1920) + len(data_2020) - 2
CI1, CI2 = stats.t.interval(alpha=0.95, loc=MU, scale=SE, df=DF)

print(f"p 値 = {p:.10f}")
print(f"t 値 = {t:.2f}")
print(f"平均値の差 = {MU:.2f}")
print(f"差の標準誤差 = {SE:.2f}")

p 値 = 0.0000000003
t 値 = -8.47
平均値の差 = 3.40
差の標準誤差 = -0.40


p 値 (有意確率) が p < 0.05 (5%) なので『2つの標本の平均値に有意差がない』としていた帰無仮説が棄却される.<br>
つまり, ｔ検定の結果として『2つの標本の平均差は(統計学的に見て)有意差があり, 2020年8月の平均気温の方が有意に温度が高い』ことが言える.<br><br>

※ ここでの有意確率 p の具体的な意味は『2つの標本がｔ検定の前提条件(無作為抽出・正規性・等分散性)を完璧に満たし,さらにそれら母集団の平均値の差が0であるとし, それら母集団から無作為に2群の標本を抽出して平均の差をとったとき, 今回のような差・あるいは今回より大きな差が観測される確率』を意味する.